# Explore here

Cargo mis dataset y veo de que está compuesto, la forma que tiene, que datos tiene, si hay nulos y duplicados.

In [22]:
import pandas as pd

datos_reviewsjuegos = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv")

datos_reviewsjuegos.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [23]:
datos_reviewsjuegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [24]:
datos_reviewsjuegos.shape

(891, 3)

In [25]:
datos_reviewsjuegos.isnull().sum()

package_name    0
review          0
polarity        0
dtype: int64

In [26]:
datos_reviewsjuegos.duplicated().sum()

np.int64(0)

In [27]:
datos_reviewsjuegos.describe()

,polarity
count,891.000000
mean,0.344557
std,0.475490
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [28]:
datos_reviewsjuegos.duplicated().sum()

np.int64(0)

In [29]:
print(datos_reviewsjuegos.loc[0, 'review'])

 privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online.


Simplifico los datos, para quitar espacios y pasarlos a minusculas.

In [30]:
datos_reviewsjuegos["review"] = datos_reviewsjuegos["review"].str.strip().str.lower()
datos_reviewsjuegos["package_name"] = datos_reviewsjuegos["package_name"].str.strip().str.lower()

Separo mis variables para las prueba de test y train.

In [53]:
from sklearn.model_selection import train_test_split

y = datos_reviewsjuegos['polarity']
X = datos_reviewsjuegos['review']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Primero juntamos X y y para train
train_data = pd.DataFrame({
    'review': X_train,
    'polarity': y_train
})

# Ahora juntamos X y y para test
test_data = pd.DataFrame({
    'review': X_test,
    'polarity': y_test
})

# Guardar como CSV
train_data.to_csv('../data/processed/train_reviews.csv', index=False)
test_data.to_csv('../data/processed/test_reviews.csv', index=False)

# Mostrar X
print('El tamaño de X train es de:' ,X_train.shape)
print('El tamaño de Y train es de:' ,y_train.shape)
print('El tamaño de X test es de:' ,X_test.shape)
print('El tamaño de y test es de:' ,y_test.shape)

El tamaño de X train es de: (712,)
El tamaño de Y train es de: (712,)
El tamaño de X test es de: (179,)
El tamaño de y test es de: (179,)


Vectorizo lo que es texto a numeros para aplicar los modelos.

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vec_model = CountVectorizer(stop_words = "english")

X_train_vec = vec_model.fit_transform(X_train).toarray()
X_test_vec = vec_model.transform(X_test).toarray()

In [33]:
y_train.shape

(712,)

Entreno mi modelo Gaussian

In [34]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train_vec, y_train)

GaussianNB()

Y sus predicciones.

In [35]:
# Predicciones
y_pred_train = model.predict(X_train_vec)
y_pred_test = model.predict(X_test_vec)

In [36]:
from sklearn.metrics import accuracy_score

print(f"GaussianNB con precisión en TRAIN: {accuracy_score(y_train, y_pred_train)}")
print(f"GaussianNB con precisión en TEST: {accuracy_score(y_test, y_pred_test)}")

GaussianNB con precisión en TRAIN: 0.9859550561797753
GaussianNB con precisión en TEST: 0.8044692737430168


In [48]:
from pickle import dump

dump(model, open("naive_bayes_GaussianNB().sav", "wb"))

Pruebo mis otros modelos MultinomialNB y BernoulliNB

In [47]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

for model_aux in [MultinomialNB(), BernoulliNB()]:
    model_aux.fit(X_train_vec, y_train)    
    # Predicciones
    y_pred_train_aux = model_aux.predict(X_train_vec)
    y_pred_test_aux = model_aux.predict(X_test_vec)
    
    # Imprimir resultados
    print(f"{model_aux.__class__.__name__} - Precisión en TRAIN: {accuracy_score(y_train, y_pred_train_aux)}")
    print(f"{model_aux.__class__.__name__} - Precisión en TEST: {accuracy_score(y_test, y_pred_test_aux)}")
    print("-" * 50)

    dump(model, open(f"naive_bayes_{model_aux}.sav", "wb"))

MultinomialNB - Precisión en TRAIN: 0.9606741573033708
MultinomialNB - Precisión en TEST: 0.8156424581005587
--------------------------------------------------
BernoulliNB - Precisión en TRAIN: 0.9199438202247191
BernoulliNB - Precisión en TEST: 0.770949720670391
--------------------------------------------------


Hago mis pruebas con los 3 modelos: GaussianNB, MultinomialNB y BernoulliNB.

Viendo los 3 modelos buscando los mejores hiperparametros, y el mejor de los 3 es BernoulliNB. 

In [38]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]  
}

model = GaussianNB()

# GridSearchCV
grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_vec, y_train)

# Mejor modelo y precisión
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)

print("Mejores hiperparámetros:", grid.best_params_)
print(f"Precisión en test con mejor modelo: {accuracy:.4f}")

Mejores hiperparámetros: {'var_smoothing': 1e-09}
Precisión en test con mejor modelo: 0.8045


In [49]:
model = GaussianNB(var_smoothing=1e-09)
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)
print(f"Precision: {accuracy_score(y_test, y_pred)}")

Precision: 0.8044692737430168


In [40]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'fit_prior': [True, False]
}

# Crear el modelo base
model = MultinomialNB()

# GridSearchCV
grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_vec, y_train)

# Mejor modelo y precisión
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)

print("Mejores hiperparámetros:", grid.best_params_)
print(f"Precisión en test con mejor modelo: {accuracy:.4f}")

Mejores hiperparámetros: {'alpha': 2.0, 'fit_prior': False}
Precisión en test con mejor modelo: 0.8212


In [51]:
model = MultinomialNB(alpha = 2.0, fit_prior = False)
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)
print(f"Precision: {accuracy_score(y_test, y_pred)}")

Precision: 0.8212290502793296


In [43]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'binarize': [0.0, 0.5, 1.0] 
}

# Crear el modelo base
model = BernoulliNB()

# GridSearchCV
grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_vec, y_train)

# Mejor modelo y precisión
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)

print("Mejores hiperparámetros:", grid.best_params_)
print(f"Precisión en test con mejor modelo: {accuracy:.4f}")

Mejores hiperparámetros: {'alpha': 0.1, 'binarize': 0.0}
Precisión en test con mejor modelo: 0.8324


In [50]:
model = BernoulliNB(alpha = 0.1, binarize= 0.0)
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)
print(f"Precision: {accuracy_score(y_test, y_pred)}")

Precision: 0.8324022346368715
